# Gromacs/pmx workflow
The workflow is divided into 6 steps

- *Step 0*: Parameterization of the ligands with the desired force field
- *Step 1*: preparation of hybrid structures and topologies. Here, ligand maps from the literature are used: this ensures equivalent comparison to the earlier calculations. Atom mapping between ligand pairs is generated with the script `atoms_to_morph.py` script. Hybrid structure and topology is generated with `make_hybrid.py` script.
- *Step 2*: Combining hybrid ligand and protein structures and topologies, placing water and complex system in the box, solvating, adding ions.
- *Step 3*: Preparation of submission scripts for simulations on an HPC. Each script can run the subsequent steps energy minimization, NVT equilibration, NPT equilibrium (production) run and non-equilibrium runs.Consists of three parts - em, eq_nvt, eq_npt (production_run). The scripts can be written for SGE and SLURM queues.
- *Step 4*: Submission of the scripts to the HPC queue.
- *Step 5*: Check of simulation states and results. Resubmits jobs if necessary.
- *Step 6*: Analysis of the non-equlibrium results. 

## Import of the workflow

In [1]:
from pmx.workflow import pmxworkflow, \
workflow1_parameterize_ligands, \
workflow2_make_perturbations, \
workflow3_solvate, \
workflow4_write_scripts, \
workflow5_submit_simulations, \
workflow6_check_simulations, \
workflow7_analyse

RDKit WARNING: [16:45:02] Enabling RDKit 2019.09.3 jupyter extensions


## Set variables in a variable container (`pmxvariables`)
Sets the following variables:
- target
- force field
- path to data
- number of replicates of the same simulation
- verbose output?

This variable container is used throughout the workflow. The ligands and predefined edges (perturbations) are read from the PLBenchmark dataset.

In [3]:
pwf = pmxworkflow.pmxvariables(target='jnk1', 
                              forcefield='openff-1.0.0.offxml', 
                              path='/home/dahahn/projects/I_OPENFORCEFIELD/benchmark_r0.1', 
                              replicates=[1], 
                              verbose=False )

--------------------------------
Step 1/6: Parameterize ligands with desired force field
-------------------------------------
The force field is chosen above in the `pmxvariables` container

In [4]:
workflow1_parameterize_ligands.coordToTopo(pwf)

=== Parameterize Ligands   jnk1                                                    ===
    - lig_17124-1
    - lig_18624-1
    - lig_18625-1
    - lig_18626-1
    - lig_18627-1
    - lig_18628-1
    - lig_18629-1
    - lig_18630-1
    - lig_18631-1
    - lig_18632-1
    - lig_18633-1
    - lig_18634-1
    - lig_18635-1
    - lig_18636-1
    - lig_18637-1
    - lig_18638-1
    - lig_18639-1
    - lig_18652-1
    - lig_18658-1
    - lig_18659-1
    - lig_18660-1


--------------------------------
Step 2/6: Preparation of hybrid structures and topologies.
-------------------------------------
This parts contain several steps (which could be merged together in the future)

In [4]:
# maps the atoms of each edge
workflow2_make_perturbations.atomsToMorph(pwf)
# creates hybrid topologies
workflow2_make_perturbations.makeHybrid(pwf)
# merges topologies
workflow2_make_perturbations.oneffFile(pwf)
# removes temporary files
workflow2_make_perturbations.cleanUp(pwf)

=== atomsToMorph           jnk1                                                    ===
    - edge_17124-1_18634-1
    - edge_18626-1_18624-1
    - edge_18636-1_18625-1
    - edge_18632-1_18624-1
    - edge_18635-1_18625-1
    - edge_18626-1_18658-1
    - edge_18639-1_18658-1
    - edge_18626-1_18625-1
    - edge_18638-1_18658-1
    - edge_18628-1_18624-1
    - edge_18631-1_18660-1
    - edge_18638-1_18634-1
    - edge_18626-1_18632-1
    - edge_18626-1_18630-1
    - edge_18631-1_18624-1
    - edge_18629-1_18627-1
    - edge_18634-1_18637-1
    - edge_18626-1_18627-1
    - edge_18631-1_18652-1
    - edge_18637-1_18631-1
    - edge_18626-1_18634-1
    - edge_18633-1_18624-1
    - edge_17124-1_18631-1
    - edge_18627-1_18630-1
    - edge_18659-1_18634-1
    - edge_18636-1_18624-1
    - edge_18626-1_18628-1
    - edge_18626-1_18660-1
    - edge_18626-1_18659-1
    - edge_18639-1_18634-1
    - edge_18635-1_18624-1
=== makeHybrid             jnk1                                             

--------------------------------
Step 3/6: Combine protein and ligand, solvate and add ions.
-------------------------------------
This part contains again several steps (which could be merged together in the future).

In [6]:
# solvate ligand
workflow3_solvate.solvateLigand(pwf)
# add ions to ligand
workflow3_solvate.addIonsLigand(pwf)

# combine protein and hybrid ligand
workflow3_solvate.combineProteinLigand(pwf)

# solvate complex
workflow3_solvate.solvateComplex(pwf)
# add ions
workflow3_solvate.addIonsComplex(pwf)

=== solvateLigand          jnk1                                                    ===
=== addIonsLigand          jnk1                                                    ===
=== Combine Protein and Ligand   jnk1                                                    ===
=== solvateComplex         jnk1                                                    ===
=== addIonsComplex         jnk1                                                    ===


--------------------------------
Step 4/6: Write submission scripts
-------------------------------------
Here, additional arguments `runtype` and `queueType` are necessary:
- `runtype` is a list of desired simulations, one or several items of 'em', 'nvt', 'eq', 'morphes'
- `queueType` defines the scheduler, and can be 'sge' or 'slurm'

In [8]:
workflow4_write_scripts.prepareSimulations(pwf, runtype=['em', 'nvt', 'eq', 'morphes'], queueType='sge')

=== Prepare Sim Scripts    jnk1     edge_17124-1_18634-1                           ===
    - water
        - stateA
            - Replicate 1
        - stateB
            - Replicate 1
    - complex
        - stateA
            - Replicate 1
        - stateB
            - Replicate 1
=== Prepare Sim Scripts    jnk1     edge_18626-1_18624-1                           ===
    - water
        - stateA
            - Replicate 1
        - stateB
            - Replicate 1
    - complex
        - stateA
            - Replicate 1
        - stateB
            - Replicate 1
=== Prepare Sim Scripts    jnk1     edge_18636-1_18625-1                           ===
    - water
        - stateA
            - Replicate 1
        - stateB
            - Replicate 1
    - complex
        - stateA
            - Replicate 1
        - stateB
            - Replicate 1
=== Prepare Sim Scripts    jnk1     edge_18632-1_18624-1                           ===
    - water
        - stateA
            - Replicate 1
   

--------------------------------
Step 5/6: Write submission scripts
--------------------------------

In [12]:
#workflow5_submit_simulations.submitSimulations(pwf)

--------------------------------
Step 6/6: Check simulations
--------------------------------

In [13]:
#workflow6_check_simulations.checkSimulations(pwf, runtype='eq', queueType='sge')

--------------------------------
Step 7/6: Analyse non-equilibrium runs
--------------------------------

In [14]:
#workflow7_analyse.analyseSimulations(pwf)